# **Inferring**
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Setup

In [11]:
from openai import OpenAI
import openai 
import os

with open("openai_key.txt", "r") as file:
    key = file.readline().strip()

client = openai.OpenAI(api_key=key)

model = 'gpt-4o'

In [12]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=key,)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## Product review text

In [13]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [14]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

The sentiment of the review is positive. The reviewer is satisfied with the lamp, the customer service, and the overall experience with the company.


In [15]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Positive


## Identify types of emotions

In [16]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

happy, satisfied, grateful, impressed, pleased


## Identify anger

In [17]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No


## Extract product and company name from customer reviews

In [18]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Item": "lamp",
    "Brand": "Lumina"
}


## Doing multiple tasks at once

In [19]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positive",
    "Anger": false,
    "Item": "lamp",
    "Brand": "Lumina"
}


## Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.Inferring Text Topics
Another application inferring by an LLM is deducing topics from a lengthy piece of text.

This time, the sample is regarding a fictitious newspaper article about a survey conducted by the government measuring the satisfaction rate of workers in government agencies. The results reveal that NASA workers had the highest satisfaction rating.

In [20]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

Five topics discussed in the article are requested from the model in a format that each item is one or two words long and in a comma-separated list. ChatGPT returns the topics as government surveys, job satisfaction, NASA, etc.

In [21]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas without numbering them.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

Government survey, Employee satisfaction, NASA, Social Security Administration, Job satisfaction


In [22]:
response.split(sep=', ')

['Government survey',
 'Employee satisfaction',
 'NASA',
 'Social Security Administration',
 'Job satisfaction']

## Make a news alert for certain topics

The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:The final sample application is about the selection of topics that a text covers, among a targeted topics list. Initially, the list of possible topics is defined:

In [23]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [24]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as a dictionay where the key is a topic and the value is 0 or 1 for each topic if it appears.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [25]:
for i in response.split(', '):
    print(i)

{
    "nasa": 1,
    "local government": 0,
    "engineering": 0,
    "employee satisfaction": 1,
    "federal government": 1
}


In [27]:
topic_dict = {
    i.split(':')[0].strip(): int(i.split(':')[1].strip().split(',')[0])
    for i in response.split(sep='}') if ':' in i
}

if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

KeyError: 'nasa'

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [31]:
#1 PROMP - MULTI TASK 
prompt  = f"""
identify the following items from the review text:
- Sentiment (positive or negative)
- Tone
- Emotions expressed (up to 5)
- Is the reviewer expressing anger? (true or false)
- Product mentioned
- Company name

Review text: '''{lamp_review}'''

Provide the answer in JSON format.
"""
print(get_completion(prompt))

{
  "Sentiment": "positive",
  "Tone": "satisfied",
  "Emotions": ["happy", "pleased", "grateful", "impressed", "satisfied"],
  "Reviewer expressing anger": false,
  "Product mentioned": "lamp",
  "Company name": "Lumina"
}


In [29]:
#2 PROMPT - ASK FOR IMPROVEMENTS AND WEKNESSES
prompt = f"""
analyze the following product review and identify:
- Two things the user liked
- One thing that could be improved
- Emotional tone of the review
- Does the customer seem likely to recommend the product? (yes/no)

Review text: '''{lamp_review}'''
"""
print(get_completion(prompt))

Two things the user liked:
1. The additional storage feature of the lamp
2. The quick response and resolution from the company's customer support

One thing that could be improved:
1. The quality control to ensure all parts are included in the package before shipping

Emotional tone of the review:
The emotional tone of the review is positive and satisfied. The user seems happy with the product and appreciative of the company's customer service.

Likely to recommend the product:
Yes, the customer seems likely to recommend the product based on their positive experience with the company and the lamp.


In [30]:
#3 PROMPT - BUSINESS ACTION ITEMS
prompt = f"""
From the following customer review, extract 3 actionable insights that a company could use to improve its product or services.

Review text: '''{lamp_review}'''
"""
print(get_completion(prompt))

1. Improve packaging and transit handling to prevent damage during shipping: The company should invest in better packaging materials or handling procedures to ensure that products arrive intact and without any defects caused during transit.

2. Enhance quality control processes: The company should review and improve their quality control processes to ensure that all products are complete and free of missing parts before shipping them out to customers. This will help prevent customer dissatisfaction and the need for replacements.

3. Provide efficient and responsive customer support: The company should continue to prioritize quick and effective customer support, as seen in the review where the customer received a replacement string and missing part promptly. This level of customer service helps build trust and loyalty among customers.


REPORT 
- Propmt precision matters: asking for a "tone" vs "list of emotions" lead to different outputs
- JSON formatting requests help ensure structured results
- LLMs can infer intent or tone reasonably well 